<a href="https://colab.research.google.com/github/waytoakash/waytoakash/blob/main/Montz_Priority_Fuzzy_Match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prerun

In [ ]:
import os
import numpy as np
import pandas as pd
import datetime
from datetime import datetime
from datetime import timedelta
import re
import time

from dateutil.relativedelta import relativedelta #
from functools import reduce

from google.colab import drive
drive.mount('/content/drive', force_remount = True)

import gspread
from google.colab import auth
auth.authenticate_user() #

from oauth2client.client import GoogleCredentials #
#gc = gspread.authorize(GoogleCredentials.get_application_default()) #

auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

from gspread_dataframe import set_with_dataframe

!pip install fuzzywuzzy
!pip install python-Levenshtein

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def changeDateFormat(row):
  return pd.to_datetime(row, errors='coerce').dt.date

def changeDateTimeFormat(row):
  return pd.to_datetime(row, errors='coerce')

def clean_text(text):
    # Remove special characters and multiple spaces, but keep numbers
    cleaned_text = re.sub(r'[^A-Za-z0-9\s]+', '', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

def get_data(spreadSheetId, rangeName):
    wb = gc.open_by_key(spreadSheetId)
    ws = wb.worksheet(rangeName)
    rows = ws.get_all_values()
    return rows
#get all data in a sheet as df and there 1st row things as the df labels.
#If sheets don't have their labels in the first row than we have to do it manuaaly as per the data
def get_data_df(spreadSheetId, rangeName):
  l = get_data(spreadSheetId, rangeName)
  ldf = pd.DataFrame.from_records(l[1:], columns = l[0])
  return ldf

def cap(text):
  return text.upper().strip()

Mounted at /content/drive


#**Pre-processing**#

In [ ]:
key = "1BBIaBIbkKuQfVYt8W8VwbRPiztg29HPuBHzW764rEwE"
range = "LeadMaster"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
df1= get_data_df(key, range)

key = "1vbQoKI8VxgXelY5LyJrJo9RwPiwyc388B6qdJVNLeTA"
range = "Leads data"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
df2= get_data_df(key, range)

NameError: name 'gc' is not defined

In [ ]:
sales = df1[['Closure City','Society Name (Sales)','Closure Date','Closure Month','year','Closure Number','Sales Product','HO Product','Gross RP Without Offset']]
sales['societycity'] = sales['Closure City']+' '+sales['Society Name (Sales)']
sales.rename(columns = {'Closure City':'City','Gross RP Without Offset':'Gross RP'}, inplace = True)

sales['City'] = sales['City'].replace(['Navi Mumbai - NMT', 'Navi Mumbai - Thane','Navi Mumbai'], 'Navi Mumbai')
sales['City'] = sales['City'].replace(['Mumbai', 'Mumbai 2'], 'Mumbai')


mntz = df2[['City','Society Name']]
mntz.rename(columns = {'Society Name':'Society Name (Sales)'}, inplace = True)
mntz['societycity'] = mntz['City']+' '+mntz['Society Name (Sales)']

mntz['City'] = mntz['City'].replace(['Navi Mumbai and Thane'], 'Navi Mumbai')
mntz['City'] = mntz['City'].replace(['Mumbai and SoBo'], 'Mumbai')
mntz['City'] = mntz['City'].replace(['Bengaluru'], 'Bangalore')

NameError: name 'df1' is not defined

In [ ]:
sales['societycity']=sales['societycity'].apply(clean_text).apply(clean_text).apply(cap)
mntz['societycity']=mntz['societycity'].apply(clean_text).apply(clean_text).apply(cap)

In [ ]:
sales.replace('',np.nan,inplace=True)
mntz.replace('',np.nan,inplace=True)

sales.dropna(inplace=True)
mntz.dropna(inplace=True)

sales.drop_duplicates('societycity',keep='first',inplace=True)
mntz.drop_duplicates('societycity',keep='first',inplace=True)

In [ ]:
#fuzzywuzzy UDF code/ use fuzz.token_set_ratio for positional consideration
def find_match(row):
    match = process.extractOne(row['societycity'], sales['societycity'], scorer=fuzz.token_sort_ratio)
    matched_name = sales.loc[match[2], 'societycity']
    closure = sales.loc[match[2], 'Closure Number']
    similarity_score = match[1]
    return pd.Series([matched_name, closure, similarity_score], index=['Matched_name', 'Closure Number', 'Similarity_Score'])


st=time.time()
# Apply the find_match function to the entire 'name' column in df1
mntz[['Matched_name', 'Closure Number', 'Similarity_Score']] = mntz.apply(find_match, axis=1)
et=time.time()

elapsed_time_minutes = (et - st) / 60

# Print the elapsed time in minutes with 4 decimal places
print(f"Elapsed time: {elapsed_time_minutes:.4f} minutes")

In [ ]:
mntz

In [ ]:
#pushing data(to the new sheet
rangeName="FuzzyMatch"
key='1vbQoKI8VxgXelY5LyJrJo9RwPiwyc388B6qdJVNLeTA'
wb = gc.open_by_key(key)
ws = wb.worksheet(rangeName)
ws.clear()
#df = pd.DataFrame(results)
set_with_dataframe(ws, mntz)